<a href="https://colab.research.google.com/github/barthas1985/Stock_indicators_Warsaw_Stock_Exchange/blob/main/Biznesrdar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import requests
#import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'}
# List for data that will be result of Looping Through Tabels.
wiersze = []
# Function that will extract/ parsing data from tabels (placed on website) accoridng to ticker name (companies listed on Warsaw Stock Exchange) and page number. 
def get_many(ticker, page):
  # Source of Data.
  url = f'https://www.biznesradar.pl/notowania-historyczne/{ticker},{page}' 
  # sending a request and return a content of the page.
  r = requests.get(url, headers = headers)
  # extracting text of content.
  soup = BeautifulSoup(r.text, 'html.parser')
  # searching for a table with requiered Data.
  tabela = soup.find('table', attrs = {'class':'qTableFull'})
  # serching for all Table rows
  ilosc_wierszy =tabela.findAll('tr')
  # looping through each row to extract data from each cell, and appending that data (from each cell) to list 'wiersze'.
  for wiersz in ilosc_wierszy:
    dane = wiersz.findAll('td')
  # Checking if Columns have any data
    if len(dane) == 0: 
        continue

    data = dane[0].getText()
    otwarcie = dane[1].getText()
    max = dane[2].getText()
    min = dane[3].getText()
    zamkniecie = dane[4].getText()
    wolumen = dane[5].getText()
    obrot = dane[6].getText()  
    wiersze.append([data, otwarcie, max, min, zamkniecie, wolumen, obrot])
  
  return
# Choose Compnies ticker name.
print('Enter A Ticker Name: ')
ticker = input()
ticker = ticker.upper()
print(ticker)


#upper_ticker = str(ticker).upper()

for x in range(1,4):
  get_many(ticker,x)



In [ ]:
# Create a dataframe.
df = pd.DataFrame(wiersze, columns = ['Data','Otwarcie','Max','Min','Zamknięcie','Wolumen','Obrót'])

# Data cleaning
df['Data'] = pd.to_datetime(df['Data']).dt.date
df['Otwarcie'] = df['Otwarcie'].astype(float)
df['Max'] = df['Max'].astype(float)
df['Min'] = df['Min'].astype(float)
df['Zamknięcie'] =df['Zamknięcie'].astype(float)
df['Wolumen'] = df['Wolumen'].str.replace(' ','')
df['Wolumen'] = df['Wolumen'].astype(int)
df['Obrót'] = df['Obrót'].str.replace(' ','')
df['Obrót'] = df['Obrót'].astype(int)


print(df.dtypes)
df.head()


In [ ]:
# Exporting data frame in csv file.
df.to_excel('lista.xlsx')

In [ ]:
df = df.set_index(pd.DatetimeIndex(df['Data'].values))


Create SMA and EMA ...( all below is a part that I am still learning:) ) 

In [ ]:
def SMA(data, period=30, column = 'Zamknięcie'):
  return data[column].rolling(window=period).mean()
def EMA(data, period = 20, column ='Zamknięcie'):
  return data[column].ewm(span = period, adjust = False).mean()



MACD Moving Average Donvergence/Divergence


In [ ]:
def MACD(data, period_long = 26, period_short = 12, period_signal = 9, column = 'Zamknięcie'):
  # Short Term Expotential moving aavergae 
  ShortEMA = EMA(data, period_short, column = column)
  # Long EMA
  LongEMA = EMA(data, period_long, column = column)
  #Calc MACD
  data['MACD'] = ShortEMA - LongEMA
  #signal line
  data['Signal_Line'] = EMA(data, period_signal, column = 'MACD')
  return data

  